## This notebooks checks the different parameters saved in each experiment file

In [1]:
import zipfile
import pickle
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy.signal import freqs,periodogram,cheby1
import numpy as np
import pprint

### Experiment

We assume a 2-qubit experiment was created and saved. To recreate the experiment you can use following code.

```

from itertools import product
from numpy import array, kron
from qmldataset import pauli_operators, run_custom_experiment

def create_experiment():
    """In this sample usage we will create a custom experiment
    """
    dimension = 4
    evolution_time = 1
    num_time_steps = 1024
    omega = [10, 12]
    dynamic_operators = [
        kron(pauli_operators[1], pauli_operators[0]),
        kron(pauli_operators[0], pauli_operators[1]),
        kron(pauli_operators[1], pauli_operators[1])]
    static_operators = [
        omega[0] * kron(pauli_operators[3], pauli_operators[0]),
        omega[1] * kron(pauli_operators[0], pauli_operators[3])]
    noise_operators = [
        kron(pauli_operators[3], pauli_operators[0]),
        kron(pauli_operators[0], pauli_operators[3])]
    measurement_operators = [
        kron(meas_op_one, meas_op_two) for meas_op_one, meas_op_two in list(
            product(pauli_operators, pauli_operators))][1:]
    initial_states_1q = [
        array([[0.5, 0.5], [0.5, 0.5]]), array([[0.5, -0.5], [-0.5, 0.5]]),
        array([[0.5, -0.5j], [0.5j, 0.5]]), array([[0.5, 0.5j], [-0.5j, 0.5]]),
        array([[1, 0], [0, 0]]), array([[0, 0], [0, 1]])
    ]
    initial_states = [
        kron(init_state_one, init_state_two) for init_state_one, init_state_two in list(
            product(initial_states_1q, initial_states_1q))]
    num_realizations = 500
    num_pulses = 5
    noise_profile = ['Type 1', 'Type 5']
    experiment_name = 'custom_2q_IX_XI_XX_N1N5Z_N1N5Z'
    num_examples = 100
    batch_size = 4
    output_location = '/home/rchakrav/progs/qmldataset_result'
    pulse_shape = "Square"

    run_custom_experiment(
        evolution_time=evolution_time,
        num_time_steps=num_time_steps,
        dimension=dimension,
        dynamic_operators=dynamic_operators,
        static_operators=static_operators,
        noise_operators=noise_operators,
        measurement_operators=measurement_operators,
        initial_states=initial_states,
        num_realizations=num_realizations,
        pulse_shape=pulse_shape,
        num_pulses=num_pulses,
        noise_profile=noise_profile,
        experiment_name=experiment_name,
        num_examples=num_examples,
        batch_size=batch_size,
        output_location=output_location
    )
```

We expect 2 files in the `output_location` specified above.


In [2]:
dataset_folder = "/home/rchakrav/progs/qmldataset_result"
undistorted_data = "S_custom_2q_IX_XI_XX_N1N5Z_N1N5Z.zip"
distorted_data = "S_custom_2q_IX_XI_XX_N1N5Z_N1N5Z_distortion.zip"

In [3]:
# let us examine example 5 of undistorted data
example_num = 4

# unzip the dataset zipfile and extract the example file
fzip  = zipfile.ZipFile(os.path.join(dataset_folder, undistorted_data), mode='r')
example_file = fzip.extract(fzip.namelist()[4])
fzip.close()

# load the example file
f     = open(example_file,  "rb")
data  = pickle.load(f)
f.close()
os.remove(example_file)

In [4]:
{"sim_parameters": simulator.get_simulation_parameters(),
                       "pulse_parameters": pulse_parameters[idx_ex:idx_ex + 1, :],
                       "time_range": simulator.time_range,
                       "pulses": pulses[idx_ex:idx_ex + 1, :],
                       "distorted_pulses": (
                           pulses[idx_ex:idx_ex + 1, :] if not distortion
                           else distorted_pulses[idx_ex:idx_ex+1, :]),
                       "expectations": np.average(expectations[idx_ex:idx_ex + 1, :], axis=1),
                       "Vo_operator": [
                           np.average(V[idx_ex:idx_ex + 1, :], axis=1) for V in vector_obs],
                       "noise": noise[idx_ex:idx_ex + 1, :],
                       "H0": hamitonian_0[idx_ex:idx_ex + 1, :],
                       "H1": hamiltonian_1[idx_ex:idx_ex + 1, :],
                       "U0": unitary_0[idx_ex:idx_ex + 1, :],
                       "UI": unitary_i[idx_ex:idx_ex + 1, :],
                       "Vo": [V[idx_ex:idx_ex + 1, :] for V in vector_obs],
                       "Eo": expectations[idx_ex:idx_ex + 1, :]
                       }

NameError: name 'simulator' is not defined

In [5]:
# Retrieve the parameters
simulation_params = data["sim_parameters"]
for param in simulation_params:
    print('-- {} --'.format(param))
    print(simulation_params[param])

-- evolution_time --
1
-- num_time_steps --
1024
-- dynamic_operators --
[array([[0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.]]), array([[0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]]), array([[0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.]])]
-- static_operators --
[array([[ 10.,   0.,   0.,   0.],
       [  0.,  10.,   0.,   0.],
       [  0.,   0., -10.,  -0.],
       [  0.,   0.,  -0., -10.]]), array([[ 12.,   0.,   0.,   0.],
       [  0., -12.,   0.,  -0.],
       [  0.,   0.,  12.,   0.],
       [  0.,  -0.,   0., -12.]])]
-- noise_operators --
[array([[ 1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0., -1., -0.],
       [ 0.,  0., -0., -1.]]), array([[ 1.,  0.,  0.,  0.],
       [ 0., -1.,  0., -0.],
       [ 0.,  0.,  1.,  0.],
       [ 0., -0.,  0., -1.]])]
-- measurement_operators --
[array([[0., 1., 0., 0.],
       [1., 0.,

In [6]:
# pulse parameter shape is [1, num_pulses, len(dynamic_operators)x3]
# for each pulse, the 3 parameters are amplitude, position, width
print("Pulse parameters shape {}".format(data["pulse_parameters"].shape))
print("Pulse Parameters value {} ".format(data["pulse_parameters"]))


Pulse parameters shape (1, 5, 9)
Pulse Parameters value [[[ 83.97713      0.12765352   0.1        -84.655045     0.12225382
     0.1        -57.59697      0.10579349   0.1       ]
  [ 78.78473      0.29389322   0.1         -8.2506895    0.27723497
     0.1        -45.178818     0.26626167   0.1       ]
  [ -3.185439     0.45079747   0.1        -84.1342       0.4290434
     0.1         15.483737     0.4184007    0.1       ]
  [-92.65461      0.6214792    0.1        -37.604713     0.5889291
     0.1        -41.77973      0.59448594   0.1       ]
  [-29.475927     0.7963937    0.1        -11.650181     0.76659817
     0.1        -59.560966     0.7715262    0.1       ]]] 


In [7]:
# both pulse and distorted_pulses contain actual values of the pulse
# shape is [1, time_steps, 1, len(dynamic_operators)x3]
print("Pulses shape {}".format(data["pulses"].shape))
print("Distorted pulses shape {} ".format(data["distorted_pulses"].shape))

Pulses shape (1, 1024, 1, 3)
Distorted pulses shape (1, 1024, 1, 3) 


In [8]:
# Noise shape (1, time_steps, num_realizations, len(noise_profile))
# For each profile, it contains the value of noise in each realization
print("Noise shape {}".format(data["noise"].shape))


Noise shape (1, 1024, 500, 2)


Vo operator shape 15
Single Vo operator shape (1, 4, 4)


In [10]:
# Expectation shape is (1, len(measurement_operators)xlen(initial_states))
# Each value is the observable of each initial state
print("Expectations shape {}".format(data["expectations"].shape))

Expectations shape (1, 540)


In [11]:
# H0 (control) shape is (1, time_steps, 1, dimension, dimension)
print("H0 shape {}".format(data["H0"].shape))
# H1 (noise) shape is (1, time_steps, num_realization, dimension, dimension)
print("H1 shape {}".format(data["H1"].shape))

H0 shape (1, 1024, 1, 4, 4)
H1 shape (1, 1024, 500, 4, 4)


In [12]:
# U0 (control) shape is (1, time_steps, 1, dimension, dimension)
# Unitary values at each time steps
print("U0 shape{}".format(data["U0"].shape))
# UI (noise) shape is (1, num_realizations, dimension, dimension)
# Total unitary at evollution time for each noise realization
print("UI shape{}".format(data["UI"].shape))

U0 shape(1, 1024, 1, 4, 4)
UI shape(1, 500, 4, 4)


In [28]:
# Vo operator is a length equal to len(measurement_operators)
# Each operator is of shape (1, dimesion, dimension)
print("Vo operator shape {}".format(len(data["Vo_operator"])))
print("Single Vo operator shape {}".format(data["Vo_operator"][0].shape))

# V0 (measurements) is a list of length equal to len(measurement_operators)
print("Vo shape {}".format(len(data["Vo"])))
# Each element of Vo is of shape (1, num_realization, dimension, dimension)
print("Single Vo {}".format(data["Vo"][0].shape))

#Vo 
print(np.average(data["Vo"][0][0], axis=0))
print(data["Vo"][0][0].shape)
print(data["Vo_operator"][0][0])

Vo operator shape 15
Single Vo operator shape (1, 4, 4)
Vo shape 15
Single Vo (1, 500, 4, 4)
[[ 0.7295772 +8.7156901e-03j -0.01439873-4.6074297e-11j
  -0.0197676 +2.3543915e-02j  0.01899632+2.4047848e-03j]
 [ 0.02596888-1.6195786e-10j  0.7295772 -8.7156920e-03j
  -0.01758901+5.5660470e-03j  0.00266077+2.7839299e-02j]
 [ 0.00266077-2.7839301e-02j  0.01899632-2.4047859e-03j
   0.67680836+1.0370284e-02j -0.01799221+2.4667257e-10j]
 [-0.01758901-5.5660461e-03j -0.0197676 -2.3543919e-02j
   0.00639141+3.1477692e-11j  0.67680836-1.0370291e-02j]]
(500, 4, 4)
[[ 0.7295772 +8.7156901e-03j -0.01439873-4.6074297e-11j
  -0.0197676 +2.3543915e-02j  0.01899632+2.4047848e-03j]
 [ 0.02596888-1.6195786e-10j  0.7295772 -8.7156920e-03j
  -0.01758901+5.5660470e-03j  0.00266077+2.7839299e-02j]
 [ 0.00266077-2.7839301e-02j  0.01899632-2.4047859e-03j
   0.67680836+1.0370284e-02j -0.01799221+2.4667257e-10j]
 [-0.01758901-5.5660461e-03j -0.0197676 -2.3543919e-02j
   0.00639141+3.1477692e-11j  0.67680836-1.0370

In [ ]:
len(pauli_operators)

In [ ]:
aa = ['A', 'B', 'C', 'D']
mo=[p for _, p in enumerate(product(aa, aa))]

In [ ]:
len(mo)

In [ ]:
540./36